# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('Sentiment140.csv')
data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [3]:
#it's a very large file, will create a sample subset to test the code

In [4]:
 data.value_counts('target')

target
0    800000
4    800000
dtype: int64

In [5]:
#there's a 50/50 split in the target, will check it's still similar after sampling the data. 

In [6]:
sample = data.sample(5000)
sample.shape

(5000, 6)

In [7]:
sample.value_counts('target')

target
4    2502
0    2498
dtype: int64

In [8]:
#split in the target is still pretty balanced, sample is OK. 

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [9]:
import re
from nltk.tokenize import word_tokenize
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/lex/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
def clean_up(s):
    no_handle = re.sub('@\w+','',s)
    noURL = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', ' ', no_handle)
    alpha = re.sub("[^a-zA-Z]+", " ", noURL)
    string=alpha.lower()
    return (string)
sample['cleaned_up'] = sample['text'].apply(clean_up)
sample.head()

,target,ids,date,flag,user,text,cleaned_up
499718,0,2186620702,Mon Jun 15 19:05:52 PDT 2009,NO_QUERY,remembermaine,@saykendrawithme I MISS THEM TOO Soooooooon. ...,i miss them too soooooooon two days
102734,0,1795179384,Thu May 14 07:18:21 PDT 2009,NO_QUERY,AprilMooMoo,Oh noooo the wound on my finger is not going t...,oh noooo the wound on my finger is not going t...
19814,0,1556938499,Sun Apr 19 01:06:56 PDT 2009,NO_QUERY,SarahSaner,"@murphygrainne I hope so, I wouldnt want too m...",i hope so i wouldnt want too many of those ki...
802083,4,1468178974,Tue Apr 07 00:08:40 PDT 2009,NO_QUERY,ForeverYours__,I'm in a David Bowie mood I feel like watchin...,i m in a david bowie mood i feel like watching...
909118,4,1751105007,Sat May 09 18:40:31 PDT 2009,NO_QUERY,g_rob,"@katintherat Send that along, lady.",send that along lady


In [11]:
#had to change the clean up function to remove the twitter handles

In [12]:
def tokenize(s):
    tokens = word_tokenize(s)
    return (tokens)
sample['tokenized'] = sample['cleaned_up'].apply(tokenize)
sample.head()

,target,ids,date,flag,user,text,cleaned_up,tokenized
499718,0,2186620702,Mon Jun 15 19:05:52 PDT 2009,NO_QUERY,remembermaine,@saykendrawithme I MISS THEM TOO Soooooooon. ...,i miss them too soooooooon two days,"[i, miss, them, too, soooooooon, two, days]"
102734,0,1795179384,Thu May 14 07:18:21 PDT 2009,NO_QUERY,AprilMooMoo,Oh noooo the wound on my finger is not going t...,oh noooo the wound on my finger is not going t...,"[oh, noooo, the, wound, on, my, finger, is, no..."
19814,0,1556938499,Sun Apr 19 01:06:56 PDT 2009,NO_QUERY,SarahSaner,"@murphygrainne I hope so, I wouldnt want too m...",i hope so i wouldnt want too many of those ki...,"[i, hope, so, i, wouldnt, want, too, many, of,..."
802083,4,1468178974,Tue Apr 07 00:08:40 PDT 2009,NO_QUERY,ForeverYours__,I'm in a David Bowie mood I feel like watchin...,i m in a david bowie mood i feel like watching...,"[i, m, in, a, david, bowie, mood, i, feel, lik..."
909118,4,1751105007,Sat May 09 18:40:31 PDT 2009,NO_QUERY,g_rob,"@katintherat Send that along, lady.",send that along lady,"[send, that, along, lady]"


In [13]:
def stem_and_lemmatize(l):
    ps = PorterStemmer()
    stemmed = [ps.stem(l) for l in l]
    lemmatizer = WordNetLemmatizer() 
    lemmatized = [lemmatizer.lemmatize(l) for l in l]
    return(lemmatized)

sample['stem_lem'] = sample['tokenized'].apply(stem_and_lemmatize)
sample.head()

,target,ids,date,flag,user,text,cleaned_up,tokenized,stem_lem
499718,0,2186620702,Mon Jun 15 19:05:52 PDT 2009,NO_QUERY,remembermaine,@saykendrawithme I MISS THEM TOO Soooooooon. ...,i miss them too soooooooon two days,"[i, miss, them, too, soooooooon, two, days]","[i, miss, them, too, soooooooon, two, day]"
102734,0,1795179384,Thu May 14 07:18:21 PDT 2009,NO_QUERY,AprilMooMoo,Oh noooo the wound on my finger is not going t...,oh noooo the wound on my finger is not going t...,"[oh, noooo, the, wound, on, my, finger, is, no...","[oh, noooo, the, wound, on, my, finger, is, no..."
19814,0,1556938499,Sun Apr 19 01:06:56 PDT 2009,NO_QUERY,SarahSaner,"@murphygrainne I hope so, I wouldnt want too m...",i hope so i wouldnt want too many of those ki...,"[i, hope, so, i, wouldnt, want, too, many, of,...","[i, hope, so, i, wouldnt, want, too, many, of,..."
802083,4,1468178974,Tue Apr 07 00:08:40 PDT 2009,NO_QUERY,ForeverYours__,I'm in a David Bowie mood I feel like watchin...,i m in a david bowie mood i feel like watching...,"[i, m, in, a, david, bowie, mood, i, feel, lik...","[i, m, in, a, david, bowie, mood, i, feel, lik..."
909118,4,1751105007,Sat May 09 18:40:31 PDT 2009,NO_QUERY,g_rob,"@katintherat Send that along, lady.",send that along lady,"[send, that, along, lady]","[send, that, along, lady]"


In [30]:
%%time

def remove_stopwords(l):
    removed = [word for word in l if not word in stopwords.words()]
    return(removed)

sample['text_processed'] = sample['stem_lem'].apply(remove_stopwords)
sample.head()

CPU times: user 2min 22s, sys: 12.9 s, total: 2min 35s
Wall time: 2min 35s


,target,ids,date,flag,user,text,cleaned_up,tokenized,stem_lem,text_processed
499718,0,2186620702,Mon Jun 15 19:05:52 PDT 2009,NO_QUERY,remembermaine,@saykendrawithme I MISS THEM TOO Soooooooon. ...,i miss them too soooooooon two days,"[i, miss, them, too, soooooooon, two, days]","[i, miss, them, too, soooooooon, two, day]","[miss, soooooooon, two, day]"
102734,0,1795179384,Thu May 14 07:18:21 PDT 2009,NO_QUERY,AprilMooMoo,Oh noooo the wound on my finger is not going t...,oh noooo the wound on my finger is not going t...,"[oh, noooo, the, wound, on, my, finger, is, no...","[oh, noooo, the, wound, on, my, finger, is, no...","[oh, noooo, wound, finger, going, heal, pulled..."
19814,0,1556938499,Sun Apr 19 01:06:56 PDT 2009,NO_QUERY,SarahSaner,"@murphygrainne I hope so, I wouldnt want too m...",i hope so i wouldnt want too many of those ki...,"[i, hope, so, i, wouldnt, want, too, many, of,...","[i, hope, so, i, wouldnt, want, too, many, of,...","[hope, wouldnt, many, kind, night, felt, unref..."
802083,4,1468178974,Tue Apr 07 00:08:40 PDT 2009,NO_QUERY,ForeverYours__,I'm in a David Bowie mood I feel like watchin...,i m in a david bowie mood i feel like watching...,"[i, m, in, a, david, bowie, mood, i, feel, lik...","[i, m, in, a, david, bowie, mood, i, feel, lik...","[david, bowie, mood, feel, like, watching, lab..."
909118,4,1751105007,Sat May 09 18:40:31 PDT 2009,NO_QUERY,g_rob,"@katintherat Send that along, lady.",send that along lady,"[send, that, along, lady]","[send, that, along, lady]","[send, along, lady]"


In [31]:
clean_sample = sample.drop(['cleaned_up','tokenized','stem_lem'],axis=1)
clean_sample.head()

,target,ids,date,flag,user,text,text_processed
499718,0,2186620702,Mon Jun 15 19:05:52 PDT 2009,NO_QUERY,remembermaine,@saykendrawithme I MISS THEM TOO Soooooooon. ...,"[miss, soooooooon, two, day]"
102734,0,1795179384,Thu May 14 07:18:21 PDT 2009,NO_QUERY,AprilMooMoo,Oh noooo the wound on my finger is not going t...,"[oh, noooo, wound, finger, going, heal, pulled..."
19814,0,1556938499,Sun Apr 19 01:06:56 PDT 2009,NO_QUERY,SarahSaner,"@murphygrainne I hope so, I wouldnt want too m...","[hope, wouldnt, many, kind, night, felt, unref..."
802083,4,1468178974,Tue Apr 07 00:08:40 PDT 2009,NO_QUERY,ForeverYours__,I'm in a David Bowie mood I feel like watchin...,"[david, bowie, mood, feel, like, watching, lab..."
909118,4,1751105007,Sat May 09 18:40:31 PDT 2009,NO_QUERY,g_rob,"@katintherat Send that along, lady.","[send, along, lady]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [32]:
flat_list = []
for sublist in clean_sample['text_processed']:
    for item in sublist:
        flat_list.append(item)
#found the loop here: https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-a-list-of-lists

In [ ]:
#Then had to look through documentation of freqdist and also how to select top 5000. 

In [34]:
from nltk.probability import FreqDist
fdist = FreqDist(flat_list)
top_5000 = fdist.most_common(5000)

top_words_only = []
for a_tuple in top_5000:
    top_words_only.append(a_tuple[0])

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [36]:
def find_features(row):
    words = row['text_processed']
    features = {}
    is_positive = (row['target'] != 0)
    for w in top_words_only:
        features[w] = (w in words)
    return (features, is_positive)

In [37]:
featuresets = list(clean_sample.apply(find_features, axis=1))

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [38]:
#featureset is still my 5000 tweets from the sample. 

5000

In [39]:
training_set = featuresets[:4000]
testing_set = featuresets[4000:]

In [40]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [41]:
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)

Classifier accuracy percent: 69.39999999999999


In [42]:
classifier.show_most_informative_features(15)

Most Informative Features
                   thank = True             True : False  =     19.1 : 1.0
                     ugh = True            False : True   =     16.6 : 1.0
                  thanks = True             True : False  =     13.5 : 1.0
                  lovely = True             True : False  =     11.5 : 1.0
                     sad = True            False : True   =      8.8 : 1.0
                headache = True            False : True   =      8.5 : 1.0
                    hate = True            False : True   =      8.1 : 1.0
                    past = True            False : True   =      7.8 : 1.0
                    hurt = True            False : True   =      7.7 : 1.0
                    miss = True            False : True   =      7.5 : 1.0
                    sick = True            False : True   =      7.5 : 1.0
                     boo = True            False : True   =      7.1 : 1.0
                    gone = True            False : True   =      7.1 : 1.0

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here